# 有限自动机
## 确定性有限自动机（DFA）
DFA:

M 是一个五元组：

$$ M = (\Sigma, Q, \delta, q_{0}, F) $$

其中， 
$ \Sigma 是输入符号的有穷集合; $

Q是状态的有限集合；

$ q_{0} \in Q 是初始状态； $

$ F是终止状态集合，F \subseteq Q；$

$ \delta 是Q与 \Sigma 的直积 Q \times \Sigma 到Q（下一个状态）的映射，它支配着有限状态控制的行为，有时也成为状态转移函数。 $


# 编辑距离
编辑距离（Edit Distance），又称Levenshtein距离，是指两个字串之间，由一个转成另一个所需的最少编辑操作次数。许可的编辑操作包括将一个字符替换成另一个字符，插入一个字符，删除一个字符。一般来说，编辑距离越小，两个串的相似度越大。

### 算法说明：
假设 $ z[j]( j\geq 1 )$ 表示含有j个字符的字符串。$ X[m] $ 为拼写错误的字符串，其长度为m，$ Y[n] $ 为与X串可能部分相同的字符串(一个候选），其长度为n。那么，给定两个串X和Y的编辑距离为 $ ed(X[m]，Y[n]) $ ，可以通过下面的循环计算：
1. 如果 $ x_{i+1}=y_{j+1} $（两个串的最后一个字母相同），则：
    $ ed(X[i+1],Y[j+1]) = ed(X[i],Y[j]) $
2. 如果 $ x_i=y_{j+1} $ ，并且 $ x_{i+1}=y_j $（最后两个字符交叉相等），则：
    $ ed(X[i+1],Y[j+1]) = 1 + min \{ ed(X[i-1],Y[j-1])，ed(X[i],Y[j+1])，ed(X[i+1],Y[j]) \} $
3. 其他情况下：
    $ ed(X[i+1],Y[j+1]) = 1 + min \{ ed(X[i],Y[j])，ed(X[i],Y[j+1])，ed(X[i+1],Y[j]) \} $
    
其中，
* $ ed(X[0],Y[j]) = j (0 \leq j \leq n) $ ；
* $ ed(X[i],Y[0]) = j (0 \leq i \leq m) $ ；
* $ ed(X[-1],Y[j]) = ed(X[i],Y[-1]) = max(m,n) $ （边界约定）；


In [3]:
# 算法实现
BOUNDARY = 100
def EditDistance(str1, str2):
    len1 = len(str1)
    len2 = len(str2)
    
    # 假定 len1 <= len2，
    if(len1 > len2):
        len1, len2, str1, str2 = len2, len1, str2, str1
    
    if (len1 == 0):
        return len2
    
    distanceMap = {}
    for i in range(len1):
        for j in range(len2):
            # 边界约定
            if i == -1 or j == -1:
                distanceMap[(i,j)] = BOUNDARY
            elif i == 0:
                distanceMap[(i,j)] = j
            elif j == 0:
                distanceMap[(i,j)] = i
            elif str1[i] == str2[j]:
                distanceMap[(i,j)] = distanceMap[(i-1, j-1)]
            elif str1[i] == str2[j-1] and str1[i-1] == str2[j]:
                distanceMap[(i,j)] = 1 + min(distanceMap[(i,j-1)], distanceMap[(i-1,j)],distanceMap[(i-2,j-2)])
            else:
                distanceMap[(i,j)] = 1 + min(distanceMap[(i,j-1)], distanceMap[(i-1,j)],distanceMap[(i-1,j-1)])

    return distanceMap[(len1-1, len2-1)]

In [4]:
# 测试编辑距离
EditDistance('repo', 'repost')

2

## 字符串被自动机识别的条件

定义一个确定的有限状态机$ R：  R = (A, Q, \delta, q_{0}, F) $

$ 其中Q表示状态，A表示输入字符集（字母表）；\delta:Q \times A \to Q；q_{0} \ in Q为其实状态；F \subseteq Q 为终止状态． $

$ 如果L \subseteq A^* 表示有限状态机R定义的语言，t>0　为编辑距离的阈值，那么，一个字符串X[m]\notin L能够被Ｒ识别的条件是存在非空集合：$

$ C = \{ Y[n] | Y[n] \in L, ed(X[m], Y[n]) \leq t \} $ 

## 剪除编辑距离（剪除距离 cut-off edit distance）

度量错误的输入串的子串（可能是局部）与候选正确串之间的最小编辑距离。

给定阈值t，寻找所有那些与输入串的编辑距离小于t的候选串。设Y是一局部候选串（拼写正确），其长度为n，X是出错的输入串，其长度为m, 令 l=max(1, n-t), u=min(m,n+t)，那么，剪除距离定义为：

$$ cut\_ed(X[m], Y[n]) = \displaystyle \min_{l \leq i \leq u} ed(X[i], Y[n]) $$



In [16]:
# 算法实现
def CutEditDistance(str1, str2, t):
    m = len(str1)
    n = len(str2)
    l = max(1, n-t)
    u = min(m, n+t)
    return min([EditDistance(str1[:i],str2) for i in range(l,u+1)])

In [20]:
# 测试剪除编辑距离
CutEditDistance('reprter','repo', 2)

1

有限自动机形成候选串Ｙ的过程构成一个有向图，因此，可以通过稍微修改图的深度优先搜索算法来实现所有Ｙ的生成过程，算法描述如下：

    // 将一个空字符串和初始节点压栈　stack
    push ((epsilon, q_0));
    while stack != null {
        // 从栈顶弹出一个部分字串Y_c和状态节点 
        pop(Y_c, q_i);
        for every q_j, a: delta(q_i, a) = q_j {
            //　扩展候选串，把字符a连接到Y_c上
            Y = concat(Y_c, a);
            if (cut_ed(X[m], Y[n]) <= t )
                //　保证剪除距离在限定的范围
                push((Y, q_j));
            if （(ed(X[m], Y[n]) <= t) and q_j in F）
                output(Y);
        }
    }


In [5]:
from IPython.core.display import HTML
HTML("""
<style>

div.cell { /* Tunes the space between cells */
margin-top:1em;
margin-bottom:1em;
}

div.text_cell_render h1 { /* Main titles bigger, centered */
font-size: 2.2em;
line-height:1.4em;
text-align: center;
}

div.text_cell_render h2 { /*  Parts names nearer from text */
margin-bottom: -0.4em;
}


div.text_cell_render { /* Customize text cells */
font-family: 'Times New Roman';
font-size:1.2em;
line-height:1.5em;
padding-left:3em;
padding-right:3em;
}
</style>
""")